# Example Usage

The `SkyScene` object is designed to give you access to many PRFs to make up a field of stars. This will rely on you having `gaiaoffline` installed and the database for `gaiaoffline` downloaded and in the right place.

!!! note "You must ensure you have `gaiaoffline` database installed"

    You should follow the [`gaiaoffline` documentation](https://christinahedges.github.io/gaiaoffline/) to ensure your database is installed correctly. This database is 30gb, and you should download the approved one from Zenodo or obtain it from a team member. You can check you have the database downloaded using
    ```python
    import gaiaoffline as go
    import os
    
    assert os.path.isfile(go.DATABASEPATH)
    ```

To create a SkyScene we three things:

1. The PRF: You should load the PRF you want to be used in the scene. You obtain the `PRF` from `pandoraaperture`
2. The WCS: The world coordinate system that we are using. This is determined by where we are pointing. You can obtain the WCS from `pandoraref`
3. The time: Because stars have proper motion we also need the time, to ensure the catalog has the stars in the right position. You should update the time based on when you're observing with Pandora.

We have convenience methods to load PRFs and WCS using `from_pointing` which will take an RA, Dec, and theta for the boresight of the telescope and will then load the right reference products. 

In [1]:
import pandoraaperture as pa
import matplotlib.pyplot as plt
import pandoraref as pr
import numpy as np
from astropy.time import Time

In [2]:
# coordinates for Kepler-10
ra, dec, theta = (285.6794224553767, 50.24130600481639, 85.1230985)

The first kind of `SkyScene` is the base class and is mostly useful for modeling VISDA full frame images.

In [ ]:
scene = pa.SkyScene.from_pointing(ra, dec, theta, time=Time.now())

In [ ]:
scene

Let's plot the scene. This models the scene using all the flux values in the catalog.

In [ ]:
scene.plot();

This looks good, let's take a look at a different kind of SkyScene. This is for ROIs on the VISDA. The ROIs are specified using their corners and their size. 

In [ ]:
rR, rC = np.mgrid[-500:500:3j, -500:500:3j]

ROI_corners = [(r + 1024 - 25, c + 1024 - 25) for r, c in zip(rR.ravel(), rC.ravel())]
nROIs = len(ROI_corners)
ROI_size = (50, 50)

In [ ]:
scene = pa.ROISkyScene.from_pointing(ra, dec, theta, time=Time.now(), nROIs=nROIs, ROI_size=ROI_size, ROI_corners=ROI_corners)

In [ ]:
scene.plot(vmax=1000);

This looks good too, let's look at the last kind of scene which is the `DispersedSkyScene`. This is primarily for working with NIRDA data.

In [ ]:
scene = pa.DispersedSkyScene.from_pointing(ra, dec, theta, time=Time.now())

In [ ]:
scene.plot(vmax=1000);

All `SkyScene` objects use a PRF, WCS and a catalog of sources to make scenes

In [ ]:
scene.prf

In [ ]:
scene.wcs

In [ ]:
scene.cat

Each of these items should be loaded for you by the `SkyScene` object. 

The object works by creating a design matrix for you

In [ ]:
scene.A(delta_pos=(0, 0))

This design matrix is a `Sparse3D` object which contains one image per source. When you dot with flux values you get the scene image.

Each `SkyScene` has an `evaluate` method which will return the image of the scene.

In [ ]:
r, c, im = scene.evaluate()

You can move this scene around with the `delta_pos` argument to shift it by a number of pixels. You should only do this for small shifts.